In [3]:
import paramiko
from os.path import exists
import os
from getpass import getpass

# Checks to see if the results.body file already exists.
# if the file does exist then the user has the option to delete it.
# the user can also ignore the warnings and append to the existing file.
if os.path.exists("results.body"):
    user_input = input("The file already exists. Enter 'y' to delete the file or any other key to keep and continue:")
    if user_input == ('y'):
        os.remove("results.body")

# Create the password prompt
thePass = getpass(prompt="Please enter your SSH password: ")

# Host Information
host = "192.168.1.17"
port = 22
username = "blaise"
password = thePass

try:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy)
    ssh.connect(host, port, username, password)

except paramiko.AuthenticationException:
    print("Authentication Failed")

In [4]:
sftp = ssh.open_sftp()

# Set fs.py to a string
script = 'fs.py'

# Concat the /tmp/ dir to the script
# Then upload the script
local_path = '/tmp/' + script
sftp.put(script, local_path)

# Run the uplaoded file
command = ['python3 /tmp/fs.py -d /usr/bin']

# Loop through the files and download them
for eachCMD in command:
    # Get the output from the command
    stdin, stdout, stderr = ssh.exec_command(eachCMD)

    # Get results from stdout
    lines = stdout.readlines()
    # print(lines)

    # Covert the list to a string
    output = ''.join(lines)

    # Write to a file
    with open('results.body', 'a') as f:
        f.write(output)


ssh.close()